1) Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bfs
import pandas as pd
import collections

## Get params

In [ ]:
filters_page = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')
filters = bfs(filters_page.text, 'html.parser')
selects = filters.findAll('select')

available_params = collections.defaultdict(dict)

for select in selects:    
    options = collections.defaultdict(list)
    
    for option in select.findAll('option'):
        if option.attrs['value'] != 'null':
            options[option.text] = option.attrs['value']
        
    available_params[select.attrs['name']] = options
  
available_params

### Making params names readable

In [ ]:
params_mapper = {
        'Unité académique' : 'ww_x_UNITE_ACAD',
        'Période académique' : 'ww_x_PERIODE_ACAD',
        'Période pédagogique' : 'ww_x_PERIODE_PEDAGO',
        'Type de semestre' : 'ww_x_HIVERETE'
    }

## Get data

In [ ]:
section = available_params[params_mapper['Unité académique']]['Informatique']
semester_type = 'null'
#bachelor_semesters = available_params[params_mapper['Période pédagogique']][['', '']]

#for year in available_params[params_mapper['Période académique']]:
#    for  in available_params[params_mapper['Période pédagogique']]:

In [ ]:
# playload represent the argument present in the get request.
# We decide top get students 
payload = {
            'ww_x_GPS' : 71297531,
            'ww_i_reportModel' : 133685247,
            'ww_i_reportModelXsl' : 133685270,
            'ww_x_UNITE_ACAD' : 249847,
            'ww_x_PERIODE_ACAD' : 942175,
            'ww_x_PERIODE_PEDAGO' : 249108,
            'ww_x_HIVERETE' : 'null'
         }


current_page = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html',params=payload)
current_page.text


In [ ]:

# link to bachelor 1 and bachelor 6
periode_pedago = [249108]

#link to school year 2007-2014 (a completer)
periode_acad   = [978181] #A completer 2013-2014 inclus

etudiant = []

for peda  in periode_pedago:
    payload['ww_x_PERIODE_PEDAGO'] = peda
    for acad in periode_acad:
        payload['ww_x_PERIODE_ACAD'] = acad
        # get the data by doing a get request
        current_page = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html',params=payload)
        # Now we use BeautifulSoup to get the data from the html result.
        # First we get all row composing the document
        soup = bfs(current_page.text, 'html.parser')
        table = soup.find("table")
        # Find all the <tr> tag pairs, skip the first one, then for each.
        for row in table.find_all('tr')[2:]:
            # Create a variable of all the <td> tag pairs in each <tr> tag pair,
            col = row.find_all('td')
            # Create a variable of the string inside 1st <td> tag pair,
            etudiant.append(col[1].string.strip())
        
# Create a variable of the value of the columns
columns = {'first_name': etudiant}

# Create a dataframe from the columns variable
df = pd.DataFrame(columns)
df.head